In [22]:
%pip install pandas
%pip install numpy
%pip install matplotlib
%matplotlib inline
%pip install nltk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('vader_lexicon') 
nltk.download('punkt')  
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords');

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/gift/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /Users/gift/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/gift/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /Users/gift/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
df = pd.read_csv("../resources/raw-dataset.csv")
df['row_id'] = df.index + 1
df.head()

,category,rating,label,text,row_id
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor...",1
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I...",2
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...,3
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i...",4
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...,5


In [4]:
# Make a copy of the df, incase we need to go back and access anything later
sub_df = df[['row_id', 'text']]
sub_df.head()

,row_id,text
0,1,"Love this! Well made, sturdy, and very comfor..."
1,2,"love it, a great upgrade from the original. I..."
2,3,This pillow saved my back. I love the look and...
3,4,"Missing information on how to use it, but it i..."
4,5,Very nice set. Good quality. We have had the s...


In [5]:
# Cleanup of the data
sub_df['text'] = sub_df['text'].str.replace("[^a-zA-Z#]", " ")
sub_df['text'] = sub_df['text'].str.casefold()
sub_df.head()

/var/folders/1q/dbl1686s2m95bg37g_s56qzc0000gn/T/ipykernel_9364/2430751107.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  sub_df['text'] = sub_df['text'].str.replace("[^a-zA-Z#]", " ")
/var/folders/1q/dbl1686s2m95bg37g_s56qzc0000gn/T/ipykernel_9364/2430751107.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['text'] = sub_df['text'].str.replace("[^a-zA-Z#]", " ")
/var/folders/1q/dbl1686s2m95bg37g_s56qzc0000gn/T/ipykernel_9364/2430751107.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

,row_id,text
0,1,love this well made sturdy and very comfor...
1,2,love it a great upgrade from the original i...
2,3,this pillow saved my back i love the look and...
3,4,missing information on how to use it but it i...
4,5,very nice set good quality we have had the s...


In [6]:
sid = SentimentIntensityAnalyzer()
words = "I like to eat candy"
scores = sid.polarity_scores(words)
print(scores)

{'neg': 0.0, 'neu': 0.545, 'pos': 0.455, 'compound': 0.3612}


In [7]:
sid = SentimentIntensityAnalyzer()
sub_df['scores'] = sub_df['text'].apply(lambda x: sid.polarity_scores(x))

/var/folders/1q/dbl1686s2m95bg37g_s56qzc0000gn/T/ipykernel_9364/2426375197.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['scores'] = sub_df['text'].apply(lambda x: sid.polarity_scores(x))


In [8]:
sub_df['neg'] = sub_df['scores'].apply(lambda x: x['neg'])
sub_df['neu'] = sub_df['scores'].apply(lambda x: x['neu'])
sub_df['pos'] = sub_df['scores'].apply(lambda x: x['pos'])
sub_df['compound'] = sub_df['scores'].apply(lambda x: x['compound'])
sub_df = sub_df.drop(columns=['scores'])

/var/folders/1q/dbl1686s2m95bg37g_s56qzc0000gn/T/ipykernel_9364/169864503.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['neg'] = sub_df['scores'].apply(lambda x: x['neg'])


In [9]:
sub_df

,row_id,text,neg,neu,pos,compound
0,1,love this well made sturdy and very comfor...,0.000,0.285,0.715,0.9558
1,2,love it a great upgrade from the original i...,0.000,0.509,0.491,0.8910
2,3,this pillow saved my back i love the look and...,0.000,0.611,0.389,0.7906
3,4,missing information on how to use it but it i...,0.079,0.642,0.279,0.7227
4,5,very nice set good quality we have had the s...,0.000,0.719,0.281,0.7397
...,...,...,...,...,...,...
40427,40428,i had read some reviews saying that this bra r...,0.042,0.758,0.201,0.9954
40428,40429,i wasn t sure exactly what it would be it is ...,0.000,0.630,0.370,0.9990
40429,40430,you can wear the hood by itself wear it with ...,0.045,0.901,0.054,0.2023
40430,40431,i liked nothing about this dress the only rea...,0.000,0.643,0.357,0.9988


In [10]:
reviews = sub_df['text'].str.lower().str.split()

In [11]:
sub_df['word'] = sub_df['text'].apply(lambda x: x.lower().split())
sub_df['total_words'] = sub_df['text'].apply(lambda x: len(x))
sub_df['unique_words'] = sub_df['word'].apply(lambda x: len(set(x)))

In [12]:
sub_df

,row_id,text,neg,neu,pos,compound,word,total_words,unique_words
0,1,love this well made sturdy and very comfor...,0.000,0.285,0.715,0.9558,"[love, this, well, made, sturdy, and, very, co...",75,11
1,2,love it a great upgrade from the original i...,0.000,0.509,0.491,0.8910,"[love, it, a, great, upgrade, from, the, origi...",80,16
2,3,this pillow saved my back i love the look and...,0.000,0.611,0.389,0.7906,"[this, pillow, saved, my, back, i, love, the, ...",67,12
3,4,missing information on how to use it but it i...,0.079,0.642,0.279,0.7227,"[missing, information, on, how, to, use, it, b...",81,16
4,5,very nice set good quality we have had the s...,0.000,0.719,0.281,0.7397,"[very, nice, set, good, quality, we, have, had...",85,16
...,...,...,...,...,...,...,...,...,...
40427,40428,i had read some reviews saying that this bra r...,0.042,0.758,0.201,0.9954,"[i, had, read, some, reviews, saying, that, th...",1694,170
40428,40429,i wasn t sure exactly what it would be it is ...,0.000,0.630,0.370,0.9990,"[i, wasn, t, sure, exactly, what, it, would, b...",1304,103
40429,40430,you can wear the hood by itself wear it with ...,0.045,0.901,0.054,0.2023,"[you, can, wear, the, hood, by, itself, wear, ...",1987,203
40430,40431,i liked nothing about this dress the only rea...,0.000,0.643,0.357,0.9988,"[i, liked, nothing, about, this, dress, the, o...",1301,103


# POS Tagging

In [13]:
import nltk
nltk.download('punkt')  # Download the Punkt tokenizer model

from nltk.tokenize import sent_tokenize
from collections import Counter

[nltk_data] Downloading package punkt to /Users/gift/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
articles = ['a', 'an', 'the']
negations = ['no', 'not', 'none', 'nobody', 'nothing', 'neither', 'nowhere', 'never', 'hardly', 'barely', 'scarcely']
auxiliary = ['am', 'is', 'are', 'was', 'were', 'be', 'being', 'been', 'will', 'would', 'shall', 'should', 'may', 'might', 'must', 'can', 'could', 'do', 'does', 'did', 'have', 'having', 'has', 'had']


In [15]:

def analyze_review_text(review_text):

    # Lists for storing each type of words
    array_noun = []
    array_adj = []
    array_verb = []
    array_adv = []
    array_pro = []
    array_pre = []
    array_con = []
    array_art = []
    array_nega = []
    array_aux = []

    for j in review_text:
        text = j
        filter_text = re.sub('[^\w\s]', '', text)
        lower_text = filter_text.lower()
        tokenized_input = lower_text.split(" ")

        unique_words = Counter(tokenized_input)
        s = " ".join(unique_words.keys())

        tokenized = sent_tokenize(s)

        for i in tokenized:
            words_list = nltk.word_tokenize(i)

            art = 0
            nega = 0
            aux = 0
            for word in words_list:
                if word in articles:
                    art += 1
                elif word in negations:
                    nega += 1
                elif word in auxiliary:
                    aux += 1

            tagged = nltk.pos_tag(words_list)
            counts = Counter(tag for word, tag in tagged)

            noun = sum([counts[i] for i in counts.keys() if 'NN' in i])
            adj = sum([counts[i] for i in counts.keys() if 'JJ' in i])
            verb = sum([counts[i] for i in counts.keys() if 'VB' in i])
            adv = sum([counts[i] for i in counts.keys() if 'RB' in i])
            pro = sum([counts[i] for i in counts.keys() if (('PRP' in i) or ('PRP$' in i) or ('WP' in i) or ('WP$' in i))])
            pre = sum([counts[i] for i in counts.keys() if 'IN' in i])
            con = sum([counts[i] for i in counts.keys() if 'CC' in i])

            array_noun.append(noun)
            array_adj.append(adj)
            array_verb.append(verb)
            array_adv.append(adv)
            array_pro.append(pro)
            array_pre.append(pre)
            array_con.append(con)
            array_art.append(art)
            array_nega.append(nega)
            array_aux.append(aux)

    print('Completed')
    return array_noun, array_adj, array_verb, array_adv, array_pro, array_pre, array_con, array_art, array_nega, array_aux

In [16]:
review_texts = sub_df['text']
POS = ['noun_count', 'adj_count', 'verb_count', 'adv_count', 'pro_count', 'pre_count', 'con_count', 'art_count', 'nega_count', 'aux_count']
values = analyze_review_text(review_texts)
i = 0
for x in POS:
    sub_df[x] = pd.Series(values[i])
    sub_df[x] = sub_df[x].fillna(0)
    sub_df[x] = sub_df[x].astype(float)
    i += 1

Completed


In [17]:
POS_percentage = ['noun_percentage', 'adj_percentage', 'verb_percentage', 'adv_percentage', 'pro_percentage', 'pre_percentage', 'con_percentage', 'art_percentage', 'nega_percentage', 'aux_percentage']
for x, y in zip(POS_percentage, POS):
    sub_df[x] = sub_df[y] / sub_df["total_words"]

# Remove Stop Words / Stemming

In [18]:
def remove_stopwords(tokenized_column):
    stops = set(stopwords.words("english"))
    res = [word for word in tokenized_column if not word in stops]
    return res if len(res) != 0 else " "

In [19]:
sub_df['to_be_train'] = sub_df.apply(lambda x: remove_stopwords(x['word']), axis=1)
sub_df.head()

,row_id,text,neg,neu,pos,compound,word,total_words,unique_words,noun_count,...,adj_percentage,verb_percentage,adv_percentage,pro_percentage,pre_percentage,con_percentage,art_percentage,nega_percentage,aux_percentage,to_be_train
0,1,love this well made sturdy and very comfor...,0.000,0.285,0.715,0.9558,"[love, this, well, made, sturdy, and, very, co...",75,11,3.0,...,0.013333,0.026667,0.026667,0.013333,0.000000,0.013333,0.000000,0.000000,0.000000,"[love, well, made, sturdy, comfortable, love, ..."
1,2,love it a great upgrade from the original i...,0.000,0.509,0.491,0.8910,"[love, it, a, great, upgrade, from, the, origi...",80,16,6.0,...,0.025000,0.025000,0.000000,0.012500,0.037500,0.000000,0.025000,0.000000,0.012500,"[love, great, upgrade, original, mine, couple,..."
2,3,this pillow saved my back i love the look and...,0.000,0.611,0.389,0.7906,"[this, pillow, saved, my, back, i, love, the, ...",67,12,5.0,...,0.000000,0.029851,0.000000,0.014925,0.014925,0.014925,0.014925,0.000000,0.000000,"[pillow, saved, back, love, look, feel, pillow]"
3,4,missing information on how to use it but it i...,0.079,0.642,0.279,0.7227,"[missing, information, on, how, to, use, it, b...",81,16,4.0,...,0.012346,0.037037,0.012346,0.012346,0.024691,0.012346,0.024691,0.000000,0.012346,"[missing, information, use, great, product, pr..."
4,5,very nice set good quality we have had the s...,0.000,0.719,0.281,0.7397,"[very, nice, set, good, quality, we, have, had...",85,16,3.0,...,0.023529,0.035294,0.035294,0.011765,0.011765,0.011765,0.011765,0.011765,0.035294,"[nice, set, good, quality, set, two, months]"


In [20]:
def stemming(tokenized_column):
    stemmer = PorterStemmer() 
    return [stemmer.stem(word).lower() for word in tokenized_column]

In [21]:
sub_df['to_be_train'] = sub_df.apply(lambda x: stemming(x['to_be_train']), axis=1)
sub_df.head()

,row_id,text,neg,neu,pos,compound,word,total_words,unique_words,noun_count,...,adj_percentage,verb_percentage,adv_percentage,pro_percentage,pre_percentage,con_percentage,art_percentage,nega_percentage,aux_percentage,to_be_train
0,1,love this well made sturdy and very comfor...,0.000,0.285,0.715,0.9558,"[love, this, well, made, sturdy, and, very, co...",75,11,3.0,...,0.013333,0.026667,0.026667,0.013333,0.000000,0.013333,0.000000,0.000000,0.000000,"[love, well, made, sturdi, comfort, love, pretti]"
1,2,love it a great upgrade from the original i...,0.000,0.509,0.491,0.8910,"[love, it, a, great, upgrade, from, the, origi...",80,16,6.0,...,0.025000,0.025000,0.000000,0.012500,0.037500,0.000000,0.025000,0.000000,0.012500,"[love, great, upgrad, origin, mine, coupl, year]"
2,3,this pillow saved my back i love the look and...,0.000,0.611,0.389,0.7906,"[this, pillow, saved, my, back, i, love, the, ...",67,12,5.0,...,0.000000,0.029851,0.000000,0.014925,0.014925,0.014925,0.014925,0.000000,0.000000,"[pillow, save, back, love, look, feel, pillow]"
3,4,missing information on how to use it but it i...,0.079,0.642,0.279,0.7227,"[missing, information, on, how, to, use, it, b...",81,16,4.0,...,0.012346,0.037037,0.012346,0.012346,0.024691,0.012346,0.024691,0.000000,0.012346,"[miss, inform, use, great, product, price]"
4,5,very nice set good quality we have had the s...,0.000,0.719,0.281,0.7397,"[very, nice, set, good, quality, we, have, had...",85,16,3.0,...,0.023529,0.035294,0.035294,0.011765,0.011765,0.011765,0.011765,0.011765,0.035294,"[nice, set, good, qualiti, set, two, month]"


In [22]:
sub_df['rating'] = df['rating']
sub_df['label'] = df['label']
sub_df['to_be_train'] = sub_df.apply(lambda x: " ".join(x['to_be_train']), axis=1)

In [23]:
sub_df.head(10)

,row_id,text,neg,neu,pos,compound,word,total_words,unique_words,noun_count,...,adv_percentage,pro_percentage,pre_percentage,con_percentage,art_percentage,nega_percentage,aux_percentage,to_be_train,rating,label
0,1,love this well made sturdy and very comfor...,0.000,0.285,0.715,0.9558,"[love, this, well, made, sturdy, and, very, co...",75,11,3.0,...,0.026667,0.013333,0.000000,0.013333,0.000000,0.000000,0.000000,love well made sturdi comfort love pretti,5.0,CG
1,2,love it a great upgrade from the original i...,0.000,0.509,0.491,0.8910,"[love, it, a, great, upgrade, from, the, origi...",80,16,6.0,...,0.000000,0.012500,0.037500,0.000000,0.025000,0.000000,0.012500,love great upgrad origin mine coupl year,5.0,CG
2,3,this pillow saved my back i love the look and...,0.000,0.611,0.389,0.7906,"[this, pillow, saved, my, back, i, love, the, ...",67,12,5.0,...,0.000000,0.014925,0.014925,0.014925,0.014925,0.000000,0.000000,pillow save back love look feel pillow,5.0,CG
3,4,missing information on how to use it but it i...,0.079,0.642,0.279,0.7227,"[missing, information, on, how, to, use, it, b...",81,16,4.0,...,0.012346,0.012346,0.024691,0.012346,0.024691,0.000000,0.012346,miss inform use great product price,1.0,CG
4,5,very nice set good quality we have had the s...,0.000,0.719,0.281,0.7397,"[very, nice, set, good, quality, we, have, had...",85,16,3.0,...,0.035294,0.011765,0.011765,0.011765,0.011765,0.011765,0.035294,nice set good qualiti set two month,5.0,CG
5,6,i wanted different flavors but they are not,0.000,1.000,0.000,0.0000,"[i, wanted, different, flavors, but, they, are...",44,8,1.0,...,0.045455,0.022727,0.000000,0.022727,0.000000,0.022727,0.022727,want differ flavor,3.0,CG
6,7,they are the perfect touch for me and the only...,0.000,0.701,0.299,0.7506,"[they, are, the, perfect, touch, for, me, and,...",89,17,4.0,...,0.011236,0.022472,0.011236,0.011236,0.022472,0.000000,0.022472,perfect touch thing wish littl space,5.0,CG
7,8,these done fit well and look great i love th...,0.000,0.482,0.518,0.9169,"[these, done, fit, well, and, look, great, i, ...",85,14,4.0,...,0.000000,0.000000,0.011765,0.011765,0.011765,0.000000,0.000000,done fit well look great love smooth edg extra,3.0,CG
8,9,great big numbers easy to read the only thi...,0.000,0.578,0.422,0.8591,"[great, big, numbers, easy, to, read, the, onl...",81,16,5.0,...,0.000000,0.000000,0.024691,0.000000,0.012346,0.000000,0.012346,great big number easi read thing like size,5.0,CG
9,10,my son loves this comforter and it is very wel...,0.000,0.559,0.441,0.8580,"[my, son, loves, this, comforter, and, it, is,...",74,16,3.0,...,0.040541,0.040541,0.000000,0.013514,0.013514,0.000000,0.027027,son love comfort well made also babi,5.0,CG


In [24]:
sub_df.to_csv('../resources/sentiment_scores.csv')